<a href="https://colab.research.google.com/github/Nazarique/Keras_redes_neurais/blob/main/sequential_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
def norm(x, lim_sup=1, lim_inf=-1):
  return lim_inf + (x - x.min(axis=0))*(lim_sup-lim_inf) / (x.max(axis=0) - x.min(axis=0))

In [ ]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
class MLP(object):

    def __init__(self, optimizer, *args, **kwargs):
        
        self.shape = args
        self.neuron = len(args)
        options = {
            'hidden_activation' : 'relu',
            'out_activation'    : 'softmax'
            }

        options.update(kwargs)
        self.inputDim = args[0]
        self.hidden_activation = options['hidden_activation']
        self.out_activation= options['out_activation']
        self.optimizer = optimizer

        self.build_model()
    
    def build_model(self):
      self.model = Sequential()
      # self.model.add(Dense(10, input_shape=(4,), activation='relu', name='fc1'))
      # self.model.add(Dense(10, activation='relu', name='fc2'))
      # self.model.add(Dense(3, activation='softmax', name='output'))
      for i in range(1, self.neuron):

        name = str(i)
        if i == 1:
          self.model.add(Dense(self.shape[i], name=name, input_shape=(self.shape[0],), activation = self.hidden_activation))
						# contruindo a camada de entrada, juntamente com os neuronios da camada escondida
        elif i == (self.neuron-1):
            self.model.add(Dense(self.shape[i], name='out_layer',activation = self.out_activation))
						# construindo a camada de saida
        else:
            self.model.add(Dense(self.shape[i], name=name, activation = self.hidden_activation))
            # contruindo camada escondidas. caso o modelo contenha mais de uma camada escondida 
                                              
      self.model.compile(optimizer=self.optimizer, loss='mean_squared_error', metrics=['accuracy']) 

      print('Neural Network Model Summary: ')
      print(self.model.summary())

    def train_model(self, trainX, trainY, valX=None, valY=None, epoch=100, batchSize=5, verbose=1):
      
      if valX==None or valY==None:
        validation_data=None
      else:
        validation_data = (valX, valY)

      self.history = self.model.fit(trainX, trainY, epochs=epoch, validation_data = validation_data, 
                                    validation_split=0.2, batch_size=batchSize, verbose=verbose)
      
    def test_model(self, testX, testY):
      
      self.score = self.model.evaluate(testX, testY, verbose=1)

# Iris

In [ ]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

iris_data = load_iris() # load the iris dataset

x = iris_data.data

x = norm(x)
y = iris_data.target.reshape(-1, 1) # Convert data to a single column

# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

# Split the data for training and testing
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)

print('Example data: ')
print(train_x[:5])
print('\nExample labels: ')
print(train_y[:5])

In [ ]:
# creating an classifier from the model:
optimizer = SGD(learning_rate=0.01, momentum=0.9)
# optimizer =  Adam(lr=0.001)

mlp = MLP(optimizer, 4, 10, 10, 3)
mlp.train_model( train_x,  train_y, epoch=200, batchSize=5, verbose=0)
mlp.test_model(test_x, test_y)

Neural Network Model Summary: 
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1 (Dense)                    (None, 10)                50        
_________________________________________________________________
2 (Dense)                    (None, 10)                110       
_________________________________________________________________
out_layer (Dense)            (None, 3)                 33        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 1ms/step - loss: 0.0171 - accuracy: 0.9667


# bank

In [ ]:
df = pd.read_csv('/content/bank-additional-full.csv', delimiter =';')
df.head(4)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [ ]:
df = MultiColumnLabelEncoder(columns = ['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week','poutcome', 'y']).fit_transform(df)

In [ ]:
x, y = df.iloc[:,0:-1], df.iloc[:,-1] 

y = np.array(y).reshape(-1, 1) # Convert data to a single column
x = norm(np.array(x))
# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)
#print(y)
# Split the data for training and testing
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)


2

In [ ]:
# creating an classifier from the model:
optimizer = SGD(learning_rate=0.01, momentum=0.9)
# optimizer =  Adam(lr=0.001)

mlp = MLP(optimizer, 20, 10, 10, 2)
mlp.train_model( train_x,  train_y, epoch=200, batchSize=5, verbose=0)
mlp.test_model(test_x, test_y)

Neural Network Model Summary: 
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1 (Dense)                    (None, 10)                210       
_________________________________________________________________
2 (Dense)                    (None, 10)                110       
_________________________________________________________________
out_layer (Dense)            (None, 2)                 22        
Total params: 342
Trainable params: 342
Non-trainable params: 0
_________________________________________________________________
None
258/258 [==============================] - 0s 1ms/step - loss: 0.0579 - accuracy: 0.9145


# wine

In [ ]:
df = pd.read_csv('/content/winequality-white.csv', delimiter =';')
df.head(4)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [ ]:
x, y = df.iloc[:,0:-1], df.iloc[:,-1] 

y = np.array(y).reshape(-1, 1) # Convert data to a single column
x = norm(np.array(x))
# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)
#print(y)
# Split the data for training and testing
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)
len(test_y[0])

7

In [ ]:
# creating an classifier from the model:
# optimizer = SGD(learning_rate=0.01, momentum=0.9)
optimizer =  Adam(lr=0.001)

mlp = MLP(optimizer, len(test_x[0]), 100, 100, len(test_y[0]))
mlp.train_model( train_x,  train_y, epoch=200, batchSize=5, verbose=0)
mlp.test_model(test_x, test_y)

Neural Network Model Summary: 
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1 (Dense)                    (None, 100)               1200      
_________________________________________________________________
2 (Dense)                    (None, 100)               10100     
_________________________________________________________________
out_layer (Dense)            (None, 7)                 707       
Total params: 12,007
Trainable params: 12,007
Non-trainable params: 0
_________________________________________________________________
None
31/31 [==============================] - 0s 1ms/step - loss: 0.0859 - accuracy: 0.6184


# Statlog

In [ ]:
train = pd.read_csv('/content/shuttle.trn', delimiter =' ', header=None)
train.head(4)

,0,1,2,3,4,5,6,7,8,9
0,50,21,77,0,28,0,27,48,22,2
1,55,0,92,0,0,26,36,92,56,4
2,53,0,82,0,52,-5,29,30,2,1
3,37,0,76,0,28,18,40,48,8,1


In [ ]:
test = pd.read_csv('/content/shuttle.trn', delimiter =' ', header=None)
test.head(4)

,0,1,2,3,4,5,6,7,8,9
0,50,21,77,0,28,0,27,48,22,2
1,55,0,92,0,0,26,36,92,56,4
2,53,0,82,0,52,-5,29,30,2,1
3,37,0,76,0,28,18,40,48,8,1


In [ ]:
x_train, y_train = df.iloc[:,0:-1], df.iloc[:,-1]
x_teste, y_teste = df.iloc[:,0:-1], df.iloc[:,-1] 

train_y = np.array(y_train).reshape(-1, 1) # Convert data to a single column
test_y = np.array(y_teste).reshape(-1, 1) # Convert data to a single column

train_x = norm(np.array(x_train))
test_x = norm(np.array(x_teste))
# One Hot encode the class labels

encoder = OneHotEncoder(sparse=False)
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [ ]:
# creating an classifier from the model:
# optimizer = SGD(learning_rate=0.01, momentum=0.9)
optimizer =  Adam(lr=0.001)

mlp = MLP(optimizer, len(test_x[0]), 100, 100, len(test_y[0]))
mlp.train_model( train_x,  train_y, epoch=200, batchSize=5, verbose=0)
mlp.test_model(test_x, test_y)

Neural Network Model Summary: 
Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1 (Dense)                    (None, 100)               1200      
_________________________________________________________________
2 (Dense)                    (None, 100)               10100     
_________________________________________________________________
out_layer (Dense)            (None, 7)                 707       
Total params: 12,007
Trainable params: 12,007
Non-trainable params: 0
_________________________________________________________________
None
154/154 [==============================] - 0s 1ms/step - loss: 0.0445 - accuracy: 0.8120


# Poker

In [ ]:
train = pd.read_csv('/content/poker-hand-training-true.data', delimiter =',', header=None)
train.head(4)

,0,1,2,3,4,5,6,7,8,9,10
0,1,10,1,11,1,13,1,12,1,1,9
1,2,11,2,13,2,10,2,12,2,1,9
2,3,12,3,11,3,13,3,10,3,1,9
3,4,10,4,11,4,1,4,13,4,12,9


In [ ]:
test = pd.read_csv('/content/poker-hand-testing.data', delimiter =',', header=None)
test.head(4)

,0,1,2,3,4,5,6,7,8,9,10
0,1,1,1,13,2,4,2,3,1,12,0.0
1,3,12,3,2,3,11,4,5,2,5,1.0
2,1,9,4,6,1,4,3,2,3,9,1.0
3,1,4,3,13,2,13,2,1,3,6,1.0


In [ ]:
x_train, y_train = df.iloc[:,0:-1], df.iloc[:,-1]
x_teste, y_teste = df.iloc[:,0:-1], df.iloc[:,-1] 

train_y = np.array(y_train).reshape(-1, 1) # Convert data to a single column
test_y = np.array(y_teste).reshape(-1, 1) # Convert data to a single column

train_x = norm(np.array(x_train))
test_x = norm(np.array(x_teste))
# One Hot encode the class labels

encoder = OneHotEncoder(sparse=False)
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [ ]:
# creating an classifier from the model:
# optimizer = SGD(learning_rate=0.01, momentum=0.9)
optimizer =  Adam(lr=0.001)

mlp = MLP(optimizer, len(test_x[0]), 100, 100, len(test_y[0]))
mlp.train_model( train_x,  train_y, epoch=200, batchSize=5, verbose=0)
mlp.test_model(test_x, test_y)

Neural Network Model Summary: 
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1 (Dense)                    (None, 100)               1200      
_________________________________________________________________
2 (Dense)                    (None, 100)               10100     
_________________________________________________________________
out_layer (Dense)            (None, 7)                 707       
Total params: 12,007
Trainable params: 12,007
Non-trainable params: 0
_________________________________________________________________
None
154/154 [==============================] - 0s 1ms/step - loss: 0.0447 - accuracy: 0.8030


# Chess (King-Rook vs. King) 

In [ ]:
df = pd.read_csv('/content/krkopt.data', delimiter =',', header=None)
df.head(4)

,0,1,2,3,4,5,6
0,a,1,b,3,c,2,draw
1,a,1,c,1,c,2,draw
2,a,1,c,1,d,1,draw
3,a,1,c,1,d,2,draw


In [ ]:
df = MultiColumnLabelEncoder(columns = [0, 2,  4,  6]).fit_transform(df)

In [ ]:
x, y = df.iloc[:,0:-1], df.iloc[:,-1] 

y = np.array(y).reshape(-1, 1) # Convert data to a single column
x = norm(np.array(x))
# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)
#print(y)
# Split the data for training and testing
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)


In [ ]:
# creating an classifier from the model:
# optimizer = SGD(learning_rate=0.01, momentum=0.9)
optimizer =  Adam(lr=0.001)

mlp = MLP(optimizer, len(test_x[0]), 100, 100, len(test_y[0]))
mlp.train_model( train_x,  train_y, epoch=200, batchSize=5, verbose=0)
mlp.test_model(test_x, test_y)

Neural Network Model Summary: 
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1 (Dense)                    (None, 100)               700       
_________________________________________________________________
2 (Dense)                    (None, 100)               10100     
_________________________________________________________________
out_layer (Dense)            (None, 18)                1818      
Total params: 12,618
Trainable params: 12,618
Non-trainable params: 0
_________________________________________________________________
None
